<a href="https://colab.research.google.com/github/jorgentak/Imersao_Alura/blob/main/Projeto_Imersao_Acoes_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
%pip -q install google-genai

In [41]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [42]:
!pip install tabulate #Instalacao de um componente para uso de tabelas

In [43]:
from tabulate import tabulate
from IPython.display import display, Markdown
import textwrap

In [44]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()



In [45]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [46]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [47]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [48]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [49]:
#################################################################
# --- Agente 1: Buscador de Ações fornecidos pelo usuário  ---- #
#################################################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Este conjunto de instruções define um agente de IA com o papel de trader especialista em análise de ações. Sua tarefa principal é usar a ferramenta de busca do Google (google_search) para encontrar quatro métricas financeiras chave para ações: valor da ação, P/L, ROE e P/VP. O agente deve priorizar a obtenção dos dados mais recentes disponíveis na B3. Caso não encontre todas as informações necessárias na data mais recente, ele deve procurar em datas anteriores até coletar todos os dados para as ações solicitadas. Para realizar a busca, o agente deve focar em fontes financeiras confiáveis, como a B3, Google Finance, Yahoo Finance e Investing.com.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [56]:
####################################################
# --- Agente 2: Complementador Sugestoes de IA --- #
####################################################
def agente_planejador(topico, lancamentos_buscados,data_de_hoje):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
          Este agente atua como um trader especialista em análise de ações IBOV. Sua principal tarefa é complementar uma lista inicial de ações (fornecida por outro agente, o "buscador") sugerindo mais 3 ações "seguras".
          Para fazer isso, ele deve:
          Usar a busca do Google para encontrar informações sobre as 3 ações iniciais (escolhidas pelo usuário) e as 3 novas sugestões.
          Coletar dados específicos para cada uma das 6 ações: valor da ação, P/L, ROE e P/VP.
          Buscar os dados mais recentes disponíveis na B3 e, se necessário, pesquisar em datas anteriores até encontrar todas as informações para todas as ações.
          Priorizar a busca de informações em fontes confiáveis como B3, Google Finance, Yahoo Finance e Investing.com.
          Incluir a data em que os valores foram extraídos.
          No final, apresentar os dados das 3 ações originais do usuário mais as 3 novas sugestões da IA.
        """,
        description="Agente com 3 sugestões de ações seguras pelo IA",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [57]:
##################################################
# --- Agente 3: Analise das ações em tabelas --- #
##################################################
def agente_analisador(topico, plano_de_analise, tablefmt='pipe'):
    analisador = Agent(
        name="agente_analisador",
        model="gemini-2.5-flash-preview-04-17", # Usado esta versao pois nao rodou com a outra
        instruction="""
            Você é um trader especializado em analisar oportunidades de investimentos em ações, use o estilo do primo rico para escrever e relatar sobre as análises, mas não se identificar.
            Criar uma tabela:
            Gerar uma tabela com informações de ações fornecidas pelo usuário e sugestões de outras IAs.
            Buscar dados: Utilizar a ferramenta Google Search para encontrar o valor da ação, P/L, ROE e P/VP para cada ação.
            Priorizar dados recentes: Buscar os dados mais recentes disponíveis na B3. Se não encontrar, pesquisar datas anteriores até obter todos os dados.
            Usar fontes confiáveis: Buscar informações em sites financeiros como B3, Google Finance, Yahoo Finance, Investing.com.
            Identificar sugestões da IA: Marcar as ações sugeridas pela IA com asterisco (*) no ticker e incluir uma nota de rodapé explicando que são sugestões da IA.
            Incluir data de extração: Adicionar a data única de extração dos dados dos valores das ações em uma observação no rodapé.
            Fazer uma análise: Analisar os pontos relevantes das ações e, após a tabela, fornecer uma conclusão final sucinta, motivadora e engajadora sobre o tema.
        """,
        description="Agente analisador de ações em forma de tabelas"
    )
    entrada_do_agente_analisador = f"Tópico: {topico}\nPlano de post: {plano_de_analise}"
    # Executa o agente
    rascunho = call_agent(analisador, entrada_do_agente_analisador)

    # Define the headers for the table
    headers = ["Ação", "Valor", "P/L", "ROE"]

    markdown_table = tabulate(topico, headers=headers, tablefmt=tablefmt)

    # Indenta o texto para exibir como um bloco de citação no Markdown (opcional)
    indented_text = textwrap.indent(markdown_table, '> ', predicate=lambda _: True)

    # Exibe o Markdown
    display(Markdown(indented_text))

    # Exemplo de uso (substitua com a saída real do seu agente)
    # Suponha que 'dados_da_tabela' é uma lista de listas e 'cabecalhos' é uma lista
    dados_da_tabela = [
    ["Ação1", "Valor1", "PL1", "ROE1"],
    ["Ação2", "Valor2", "PL2", "ROE2"],
    ["Ação3", "Valor3", "PL3", "ROE3"],
    ["Ação4", "Valor4", "PL4", "ROE4"],
    ["Ação5", "Valor5", "PL5", "ROE5"],
    ["Ação6", "Valor6", "PL6", "ROE6"]
    ]
    cabecalhos = ["Ação", "Valor", "P/L", "ROE"]


    # print("--- 📝 Resultado Formatado em Tabela Markdown ---")
    # The function to_markdown_table is not defined in the provided code.
    # If you intend to use this, you need to define it.
    # For now, we will comment it out to avoid another NameError.
    # to_markdown_table(dados_da_tabela, cabecalhos)
    print("--------------------------------------------------------------")

    return rascunho

In [52]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-flash-preview-04-17",   # Usado esta versao pois nao rodou com a outra
        instruction="""
            Você é um consultor financeiro de ações e revisor de conteúdo meticuloso, especializado tabelas e trader.
            Revise o rascunho sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de conteúdo"
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [58]:
data_de_hoje = date.today().strftime("%d/%m/%Y")
contador = 0 # Controlador de repetição enquanto não digitar as 3 Ações
vetorAcao = []

print("\n--- 🔍 Resultado do Agente 1 (Buscador) ---\n")
print("🚀 Iniciando o Sistema de Análise dos dados de 3 Ações utilizando Agentes 🚀")

while contador < 3: # Repeticao enquanto nao digitar 3 Ações ---

# Inserir lógica do sistema de agentes ################################################
    # --- Obter ao Tópico do Usuário ---
    topico = input("Por favor, digite a ações que você quer analisar: ").upper()
    print(f"Maravilha! Vamos então\ analisar a ação {topico} , [{contador+1}/3]")# Foi colocado o contador mais para informar o usuário em qual passo de 3 ele está
    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    vetorAcao.append(topico)
    #print ('vetorAcao')
    #display(to_markdown(lancamentos_buscados)) - Foi inibido este campos para o visual para o usuário ficar clean!
    print('\n')
    contador = contador + 1

print ("Bora lá! Realizamos o primeiro passo que é coletar as informacões das 3 Ações!")
plano_de_analise = agente_planejador(vetorAcao, lancamentos_buscados,data_de_hoje)
print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
print ("O Agente Trader incluiu mais 3 sugestões de ações que vale a pena conferir!")
print("----------------------------------------------------------------------------")

print ("Já realizamos o levantamento das 3 Ações que voce informou mais 3 sugestoes feito pelo IA!")
plano_tabela = agente_analisador(vetorAcao, plano_de_analise)
print("\n--- 📝 Resultado do Agente 3 (Analisador) ---\n")

print ("O Agente analisou e gerou uma tabela para uma visualização simplificada e de fácil entendimento!")
display(to_markdown(plano_tabela))
print("-------------------------------------------------------------------------------------------------")

print ("O Agente revisará o conteúdo e dará um feedback do conteúdo gerado!")
texto_revisado = agente_revisor(vetorAcao, plano_tabela)
print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
display(to_markdown(texto_revisado))
print("---------------------------------------------------------------------")







--- 🔍 Resultado do Agente 1 (Buscador) ---

🚀 Iniciando o Sistema de Análise dos dados de 3 Ações utilizando Agentes 🚀
Por favor, digite a ações que você quer analisar: VIVA3
Maravilha! Vamos então\ analisar a ação VIVA3 , [1/3]


Por favor, digite a ações que você quer analisar: VALE3
Maravilha! Vamos então\ analisar a ação VALE3 , [2/3]


Por favor, digite a ações que você quer analisar: MGLU3
Maravilha! Vamos então\ analisar a ação MGLU3 , [3/3]


Bora lá! Realizamos o primeiro passo que é coletar as informacões das 3 Ações!

--- 📝 Resultado do Agente 2 (Planejador) ---

O Agente Trader incluiu mais 3 sugestões de ações que vale a pena conferir!
----------------------------------------------------------------------------
Já realizamos o levantamento das 3 Ações que voce informou mais 3 sugestoes feito pelo IA!


> |    | Ação   | Valor   | P/L   |   ROE |
> |:---|:-------|:--------|:------|------:|
> | V  | I      | V       | A     |     3 |
> | V  | A      | L       | E     |     3 |
> | M  | G      | L       | U     |     3 |

--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Analisador) ---

O Agente analisou e gerou uma tabela para uma visualização simplificada e de fácil entendimento!


> Olha só, que beleza! Recebi o seu sinal, e o mercado está sempre cheio de oportunidades para quem sabe onde olhar. Você mandou a lista com MGLU3, VIVA3 e VALE3, e ainda pediu para dar umas sugestões de 'peso pesado' que a inteligência artificial soprou no seu ouvido. ITUB4, EGIE3 e BBSE3, é isso?
> 
> Excelente! Analisar esses nomes é crucial para entender diferentes dinâmicas de mercado. Temos desde varejo, mineração, joias, até o setor financeiro, energia e seguros. Uma mistura interessante!
> 
> Vamos direto ao ponto. O que realmente importa são os fundamentos. Eu fui a campo digital, mergulhei nos dados mais frescos da bolsa brasileira para trazer a fotografia atual dessas empresas. Lembre-se, meu amigo, conhecimento é poder, especialmente no mercado financeiro.
> 
> Peguei os dados mais recentes disponíveis na B3 e nos principais portais de finanças. Aqui está o que encontrei:
> 
> | Ação    | Valor da Ação (R$) | P/L   | ROE (%) | P/VP   |
> | :------ | :----------------- | :----- | :------ | :------ |
> | MGLU3   | 11,53              | -49,82 | -7,51   | 1,18    |
> | VIVA3   | 20,14              | 9,44   | 21,74   | 2,06    |
> | VALE3   | 61,24              | 9,84   | 13,75   | 1,40    |
> | ITUB4\* | 32,08              | 8,71   | 21,51   | 1,84    |
> | EGIE3\* | 43,16              | 9,84   | 25,12   | 2,64    |
> | BBSE3\* | 33,44              | 8,42   | 74,91   | 6,31    |
> 
> \* Ações sugeridas com base em análise de inteligência artificial.
> ¹ Dados extraídos em 06 de junho de 2024.
> 
> **Análise dos Fundamentos:**
> 
> Olhando para esses números, algumas coisas saltam aos olhos, e é aqui que a gente começa a separar o joio do trigo, ou melhor, as oportunidades dos riscos:
> 
> *   **MGLU3:** Percebeu o P/L e o ROE negativos? Isso indica que a empresa, neste momento, está apresentando prejuízo. O P/VP acima de 1 mostra que o mercado a precifica acima do seu valor patrimonial, mesmo com resultados negativos. É uma situação que exige muita cautela e um olhar atento para a recuperação.
> *   **VIVA3:** Um P/L e P/VP dentro de um patamar razoável e um ROE bem interessante acima de 20%. Sinal de que a empresa tem conseguido gerar valor e lucro de forma eficiente sobre o seu patrimônio. Parece uma operação saudável.
> *   **VALE3:** O P/L está na casa dos 9, indicando que o preço da ação é cerca de 9 vezes o lucro anual por ação. O ROE em 13,75% mostra uma rentabilidade decente sobre o patrimônio, e o P/VP em 1,40 sinaliza que ela está sendo negociada não muito acima do seu valor de livro. É uma gigante, claro, mas sujeita à volatilidade do preço das commodities.
> *   **ITUB4\*:** O Itaú, um colosso do setor financeiro. P/L abaixo de 9 e um ROE acima de 21% são números que mostram a força e a eficiência do banco em gerar lucro. O P/VP próximo de 1.84 indica que o mercado paga um prêmio por essa solidez e rentabilidade. Um player tradicional que costuma trazer estabilidade.
> *   **EGIE3\*:** No setor de energia, a Engie se destaca com um P/L e ROE saudáveis (ambos próximos de 9.8 e 25% respectivamente). O P/VP de 2,64 mostra que o mercado reconhece a qualidade dos ativos e a previsibilidade das receitas do setor, precificando-a acima do valor patrimonial. Setor resiliente e com contratos de longo prazo.
> *   **BBSE3\*:** Aqui temos um ponto fora da curva no ROE: impressionantes 74,91%! Isso indica uma altíssima eficiência em transformar patrimônio em lucro. O P/L na casa dos 8,42 é atrativo. No entanto, note o P/VP elevado (6,31). Isso significa que o mercado precifica a BB Seguridade mais de 6 vezes o seu valor patrimonial. É a precificação da alta rentabilidade e do modelo de negócio capital-leve.
> 
> **Conclusão Final:**
> 
> Analisar esses números é o primeiro passo para construir uma carteira vencedora. Cada uma dessas empresas opera em um setor diferente e apresenta dinâmicas próprias. Algumas mostram solidez e eficiência nos lucros (VIVA3, ITUB4, EGIE3, BBSE3), outras enfrentam desafios momentâneos (MGLU3), e há aquelas que navegam no ritmo das commodities (VALE3).
> 
> O segredo não está em seguir um "guru" ou uma sugestão cega, mas sim em **entender** o que você está comprando. Use esses dados como um mapa, não como o destino final. A verdadeira jornada é a sua, a de construir um patrimônio sólido e buscar a sua liberdade financeira. Continue estudando, continue analisando, e tome suas decisões com inteligência e estratégia. O mercado recompensa a preparação!


-------------------------------------------------------------------------------------------------
O Agente revisará o conteúdo e dará um feedback do conteúdo gerado!

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


---------------------------------------------------------------------
